In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, TimeDistributed
num_encoder_tokens = 71
num_decoder_tokens = 93

# Esto es donde guardará el contexto
latent_dim = 256  # Latent dimensionality of the encoding space.

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens), name="Encoder_Inputs") #num_encoder_tokens es la cantidad de features a la entrada
encoder = LSTM(latent_dim, return_state=True, return_sequences=False, name="Encoder_LSTM")
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens), name="Dencoder_Inputs") #num_decoder_tokens es la cantidad de features a la entrada del decoder
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the 
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True, name="Decoder_LSTM")
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='Model_Output')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.summary()

Using TensorFlow backend.


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
Encoder_Inputs (InputLayer)      (None, None, 71)      0                                            
____________________________________________________________________________________________________
Dencoder_Inputs (InputLayer)     (None, None, 93)      0                                            
____________________________________________________________________________________________________
Encoder_LSTM (LSTM)              [(None, 256), (None,  335872      Encoder_Inputs[0][0]             
____________________________________________________________________________________________________
Decoder_LSTM (LSTM)              [(None, None, 256), ( 358400      Dencoder_Inputs[0][0]            
                                                                   Encoder_LSTM[0][1]      

## Encoder

La diferencia entre **encoder_outputs** y **state_h** es que cuando **return_state=True**, **encoder_outputs** devuelve la secuencia mientras que **state_h** devuelve solo el estado del ultimo timestep

## Decoder

In [16]:
decoder_lstm.get_output_at(0)

[<tf.Tensor 'Decoder_LSTM_2/transpose_1:0' shape=(?, ?, 256) dtype=float32>,
 <tf.Tensor 'Decoder_LSTM_2/while/Exit_2:0' shape=(?, 256) dtype=float32>,
 <tf.Tensor 'Decoder_LSTM_2/while/Exit_3:0' shape=(?, 256) dtype=float32>]

decoder_outputs ya esta TimeDistributed